In [1]:
spark

# Initial Setup (Install and import necessary packages)

In [2]:
!pip install gcsfs
!pip install openpyxl

In [3]:
import pandas as pd
import gcsfs
import calendar
from google.cloud import storage
from google.cloud.exceptions import NotFound
from pyspark.sql import Row
from pyspark.sql.functions import col, lower, instr, date_format, split, rand, regexp_replace, trim, max
from pyspark.sql.functions import monotonically_increasing_id # virtually the same as factorize() from pandas.

In [4]:
# Define the bucket we will save to. Many areas will reference this variable later
bucket = "ppp-loans-bucket"

# Read clean data into appropriate dataframes

In [5]:
ppp_df = spark.read.parquet("gs://ppp-loans-bucket/cleaned/part-*")#.sample(fraction=0.001,seed=42)

In [6]:
ppp_df.count()

10406068

In [7]:
fs = gcsfs.GCSFileSystem(project='cis4400-group-project')
with fs.open("ppp-loans-bucket/cleaned/GDP.csv") as f:
    gdp_df = pd.read_csv(f)
gdp_df

,GeoFIPS,GeoName,Region,TableName,LineCode,IndustryClassification,Description,Unit,2017,2018,2019,2020,2021,2022
0,1000,Alabama,5.0,CAGDP1,1,...,Real GDP (thousands of chained 2017 dollars),Thousands of chained 2017 dollars,216615470,220808767,224944577,222081439,231892626,235807320
1,1000,Alabama,5.0,CAGDP1,2,...,Chain-type quantity indexes for real GDP,Quantity index,100,101.936,103.845,102.523,107.053,108.86
2,1000,Alabama,5.0,CAGDP1,3,...,Current-dollar GDP (thousands of current dolla...,Thousands of dollars,216615470,226263784,234526408,235118280,257986516,281569005
3,1001,"Autauga, AL",5.0,CAGDP1,1,...,Real GDP (thousands of chained 2017 dollars),Thousands of chained 2017 dollars,1762558,1787534,1730861,1722438,1727818,1929264
4,1001,"Autauga, AL",5.0,CAGDP1,2,...,Chain-type quantity indexes for real GDP,Quantity index,100,101.417,98.202,97.724,98.029,109.458
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9526,97000,Rocky Mountain,7.0,CAGDP1,2,...,Chain-type quantity indexes for real GDP,Quantity index,100,104.559,109.552,109.329,116.661,119.324
9527,97000,Rocky Mountain,7.0,CAGDP1,3,...,Current-dollar GDP (thousands of current dolla...,Thousands of dollars,681310123,730567674,776281078,781272363,880142487,974682556
9528,98000,Far West,8.0,CAGDP1,1,...,Real GDP (thousands of chained 2017 dollars),Thousands of chained 2017 dollars,3797440495,3956948041,4108525822,4048649569,4342903004,4385657757
9529,98000,Far West,8.0,CAGDP1,2,...,Chain-type quantity indexes for real GDP,Quantity index,100,104.2,108.192,106.615,114.364,115.49


In [8]:
fs = gcsfs.GCSFileSystem(project='cis4400-group-project')
with fs.open('ppp-loans-bucket/cleaned/NAICS.csv') as f:
    naics_df = pd.read_csv(f)
naics_df

,2022 NAICS US Code,2022 NAICS US Title,industry_type
0,11,"Agriculture, Forestry, Fishing and Hunting","Agriculture, Forestry, Fishing and Hunting"
1,111,Crop Production,"Agriculture, Forestry, Fishing and Hunting"
2,1111,Oilseed and Grain Farming,"Agriculture, Forestry, Fishing and Hunting"
3,11111,Soybean Farming,"Agriculture, Forestry, Fishing and Hunting"
4,111110,Soybean Farming,"Agriculture, Forestry, Fishing and Hunting"
...,...,...,...
2120,9281,National Security and International Affairs,Public Administration
2121,92811,National Security,Public Administration
2122,928110,National Security,Public Administration
2123,92812,International Affairs,Public Administration


# Checking columns

In [9]:
ppp_df.columns

['LoanNumber',
 'DateApproved',
 'SBAOfficeCode',
 'ProcessingMethod',
 'BorrowerName',
 'BorrowerAddress',
 'BorrowerCity',
 'BorrowerState',
 'BorrowerZip',
 'LoanStatusDate',
 'LoanStatus',
 'Term',
 'SBAGuarantyPercentage',
 'InitialApprovalAmount',
 'CurrentApprovalAmount',
 'UndisbursedAmount',
 'FranchiseName',
 'ServicingLenderLocationID',
 'ServicingLenderName',
 'ServicingLenderAddress',
 'ServicingLenderCity',
 'ServicingLenderState',
 'ServicingLenderZip',
 'RuralUrbanIndicator',
 'HubzoneIndicator',
 'LMIIndicator',
 'BusinessAgeDescription',
 'ProjectCity',
 'ProjectCountyName',
 'ProjectState',
 'ProjectZip',
 'CD',
 'JobsReported',
 'NAICSCode',
 'Race',
 'Ethnicity',
 'PAYROLL_PROCEED',
 'BusinessType',
 'OriginatingLenderLocationID',
 'OriginatingLender',
 'OriginatingLenderCity',
 'OriginatingLenderState',
 'Gender',
 'Veteran',
 'NonProfit',
 'ForgivenessAmount',
 'ForgivenessDate',
 'forgiveness_date',
 'date_approved',
 'loan_status_date']

In [10]:
gdp_df.columns

Index(['GeoFIPS', 'GeoName', 'Region', 'TableName', 'LineCode',
       'IndustryClassification', 'Description', 'Unit', '2017', '2018', '2019',
       '2020', '2021', '2022'],
      dtype='object')

In [11]:
naics_df.columns

Index(['2022 NAICS US Code', '2022 NAICS US Title', 'industry_type'], dtype='object')

# Reformatting

In [12]:
# Changes a list of columns to a certain datatype. This is for PySpark dataframes.
# You give it... 
#  1. The dataframe you want to modify
#  2. The list of strings for the name of columns you want to be modified
#  3. The datetype you want to modify them to

def change_cols_to_type(df, list_of_columns, data_type):
    for column in list_of_columns:
        df = df.withColumn(column, df[column].cast(data_type))
    return df

In [13]:
ppp_df_cols_to_float = ["InitialApprovalAmount", "CurrentApprovalAmount",
                "UndisbursedAmount", "ForgivenessAmount", 
                 "SBAGuarantyPercentage", "JobsReported"]
ppp_df_cols_to_long = ["ServicingLenderLocationID", "OriginatingLenderLocationID"]

ppp_df = change_cols_to_type(ppp_df, ppp_df_cols_to_float, "float")
ppp_df = change_cols_to_type(ppp_df, ppp_df_cols_to_long, "long")

#naics_df['2022 NAICS US Code'] = naics_df['2022 NAICS US Code'].astype(int)

ppp_df.printSchema()

root
 |-- LoanNumber: long (nullable = true)
 |-- DateApproved: string (nullable = true)
 |-- SBAOfficeCode: integer (nullable = true)
 |-- ProcessingMethod: string (nullable = true)
 |-- BorrowerName: string (nullable = true)
 |-- BorrowerAddress: string (nullable = true)
 |-- BorrowerCity: string (nullable = true)
 |-- BorrowerState: string (nullable = true)
 |-- BorrowerZip: string (nullable = true)
 |-- LoanStatusDate: string (nullable = true)
 |-- LoanStatus: string (nullable = true)
 |-- Term: string (nullable = true)
 |-- SBAGuarantyPercentage: float (nullable = true)
 |-- InitialApprovalAmount: float (nullable = true)
 |-- CurrentApprovalAmount: float (nullable = true)
 |-- UndisbursedAmount: float (nullable = true)
 |-- FranchiseName: string (nullable = true)
 |-- ServicingLenderLocationID: long (nullable = true)
 |-- ServicingLenderName: string (nullable = true)
 |-- ServicingLenderAddress: string (nullable = true)
 |-- ServicingLenderCity: string (nullable = true)
 |-- Servi

# Function that creates ID's for columns in a Spark Dataframe

In [14]:
def create_ids(df,id_column_name=None):
    #Transforms PySpark dataframe
    if isinstance(df, PySparkDataFrame):
        print("Transforming PySpark DataFrame...")
        
        if id_column_name is None:
            id_column_name = "id"
        
        # Extract the original schema
        original_schema = df.schema

        # Add an ID column using zipWithIndex
        df_with_index = (
            df.rdd
            .zipWithIndex()  # Add an index to each row
            .map(lambda x: Row(**dict(x[0].asDict(), id=x[1])))  # Add 'id' to each row
        )

        # Define the new schema with the ID column added
        new_schema = original_schema.add(id_column_name, "long")

        # Create a new DataFrame with the updated schema
        df_with_index = spark.createDataFrame(df_with_index, schema=new_schema)
        
        print("Completed transforming PySpark DataFrame.")
        return df_with_index
    
    #No valid dataframe found
    else:
        print("ERROR: INVALID DATAFRAME - NO PROCEDURE APPLIED.")
        print("Did you try inputting a PySpark Dataframe?")

# Function to turn column names to snake_case

In [15]:
#Turns any columns that has TheseColumnsNames to these_column_names
import re
from pyspark.sql import DataFrame as PySparkDataFrame
def name_to_snake_case(name):
    # Add underscores before capital letters, then convert to lowercase
    return re.sub(r'(?<!^)(?=[A-Z])', '_', name).lower()

#Takes in any type of dataframe and turns its columns into snake_case
def df_to_snake_case(df):
    
    #Transforms PySpark dataframe
    if isinstance(df, PySparkDataFrame):
        print("Transforming PySpark DataFrame...")
        # Rename all columns to snake_case
        snake_case_columns = [name_to_snake_case(col) for col in df.columns]
        df_snake_case = df.toDF(*snake_case_columns)
        print("Completed transforming PySpark DataFrame.")
        return df_snake_case
    
    #Transforms Pandas dataframe
    elif isinstance(df, pd.DataFrame):
        print("Transforming Pandas DataFrame...")
        # Rename all columns to snake_case
        df.columns = [name_to_snake_case(col) for col in df.columns]
        print("Completed transforming Pandas DataFrame.")
        return df
    
    #No valid dataframe found
    else:
        print("ERROR: INVALID DATAFRAME - NO PROCEDURE APPLIED")


In [16]:
# initialize array. Append all dims and tables here. At the end, we loop through this list to save all dataframes.
tables = []

In [17]:
# Function to check how unique a column is. This will be used to verify all ID's in an ID column are unique.
def check_uniqueness(df, col_name):
    if isinstance(df, PySparkDataFrame):
        total_count = df.count()
        distinct_count = df.select(col_name).distinct().count()
    elif isinstance(df, pd.DataFrame):
        total_count = len(df)
        distinct_count = df[col_name].nunique()

    if total_count == distinct_count:
        print(f"The column '{col_name}' contains all unique values.")
    else:
        print(f"The column '{col_name}' contains duplicates. Total rows: {total_count}, Distinct rows: {distinct_count}")

# Dim Borrower

In [18]:
#Create Dim
dim_borrower = ppp_df.select("LoanNumber","BorrowerName","BorrowerAddress",
                            "BorrowerCity","BorrowerState","BorrowerZip",
                            "FranchiseName","BusinessAgeDescription",
                            "Race","Ethnicity","BusinessType","Gender",
                            "Veteran","NonProfit")

#Fix Names
dim_borrower = dim_borrower.withColumnRenamed("LoanNumber","borrower_id")
dim_borrower = df_to_snake_case(dim_borrower)

#Remove duplicates
dim_borrower = dim_borrower.distinct()

#Add to tables
tables.append([dim_borrower,"dim_borrower"])

#Show
dim_borrower.show()
dim_borrower.columns

Transforming PySpark DataFrame...
Completed transforming PySpark DataFrame.


+-----------+--------------------+--------------------+-------------+--------------+------------+--------------+------------------------+----------+--------------------+--------------------+------------+-----------+----------+
|borrower_id|       borrower_name|    borrower_address|borrower_city|borrower_state|borrower_zip|franchise_name|business_age_description|      race|           ethnicity|       business_type|      gender|    veteran|non_profit|
+-----------+--------------------+--------------------+-------------+--------------+------------+--------------+------------------------+----------+--------------------+--------------------+------------+-----------+----------+
| 9647258509|NEW MADISON AVENU...|    1 Silvermine Rdg|      Norwalk|            CT|  06850-1042|          NULL|    Existing or more ...|Unanswered|   Unknown/NotStated|Limited  Liabilit...|  Unanswered| Unanswered|      NULL|
| 7781477805|HOMETECH SUNROOMS...|       499 berlin st|  Southington|            CT|  06489-

['borrower_id',
 'borrower_name',
 'borrower_address',
 'borrower_city',
 'borrower_state',
 'borrower_zip',
 'franchise_name',
 'business_age_description',
 'race',
 'ethnicity',
 'business_type',
 'gender',
 'veteran',
 'non_profit']

# Dim SBA office

In [19]:
#Create Dim
dim_SBA_office = ppp_df.select("SBAOfficeCode").distinct().toPandas()

#Fix Names
dim_SBA_office = df_to_snake_case(dim_SBA_office)
dim_SBA_office.rename(columns={'s_b_a_office_code':'SBA_office_code'}, inplace=True)

#Remove duplicates
dim_SBA_office = dim_SBA_office.drop_duplicates()

#Add to tables
tables.append([dim_SBA_office,"dim_SBA_office"])

#Show
dim_SBA_office

Transforming Pandas DataFrame...
Completed transforming Pandas DataFrame.


,SBA_office_code
0,914
1,811
2,474
3,299
4,358
...,...
71,130
72,341
73,172
74,721


# Dim Processing Method

In [20]:
#Create Dim
dim_processing_method = ppp_df.select("ProcessingMethod").distinct().toPandas()

#Fix Names
dim_processing_method = df_to_snake_case(dim_processing_method)

#Remove duplicates
dim_processing_method = dim_processing_method.drop_duplicates()

#Add to tables
tables.append([dim_processing_method,"dim_processing_method"])

#Show
dim_processing_method

Transforming Pandas DataFrame...
Completed transforming Pandas DataFrame.


,processing_method
0,PPP
1,PPS


# Dim Originating Lender

In [21]:
#Create Dim
dim_originating_lender = ppp_df.select("OriginatingLenderLocationID",
                                      "OriginatingLender","OriginatingLenderCity",
                                      "OriginatingLenderState")

#Fix Names
dim_originating_lender = df_to_snake_case(dim_originating_lender)
dim_originating_lender = dim_originating_lender.withColumnRenamed("originating_lender_location_i_d","originating_lender_location_id")

#Remove duplicates
dim_originating_lender = dim_originating_lender.distinct()

#Add to tables
tables.append([dim_originating_lender,"dim_originating_lender"])

#Show
dim_originating_lender.show()

Transforming PySpark DataFrame...
Completed transforming PySpark DataFrame.


+------------------------------+--------------------+-----------------------+------------------------+
|originating_lender_location_id|  originating_lender|originating_lender_city|originating_lender_state|
+------------------------------+--------------------+-----------------------+------------------------+
|                         25001| First Heritage Bank|             Shenandoah|                      IA|
|                        532784|        People Trust|            Little Rock|                      AR|
|                        448174|     Millennium Bank|            DES PLAINES|                      IL|
|                        238179|Cedar Rapids Bank...|           Cedar Rapids|                      IA|
|                        124107|       Highland Bank|                    Ely|                      MN|
|                        119777|      Builtwell Bank|                 Dalton|                      GA|
|                        123532|           NBKC Bank|                Leaw

# Dim Servicing Lender

In [22]:
#Create Dim
dim_servicing_lender = ppp_df.select('ServicingLenderLocationID', 'ServicingLenderName',
                                     'ServicingLenderAddress', 'ServicingLenderCity', 
                                     'ServicingLenderState','ServicingLenderZip')

#Fix Names
dim_servicing_lender = df_to_snake_case(dim_servicing_lender)
dim_servicing_lender = dim_servicing_lender.withColumnRenamed("servicing_lender_location_i_d","servicing_lender_location_id")

#Remove duplicates
dim_servicing_lender = dim_servicing_lender.distinct()

#Add to tables
tables.append([dim_servicing_lender,"dim_servicing_lender"])

#Show
dim_servicing_lender.show()

Transforming PySpark DataFrame...
Completed transforming PySpark DataFrame.


+----------------------------+---------------------+------------------------+---------------------+----------------------+--------------------+
|servicing_lender_location_id|servicing_lender_name|servicing_lender_address|servicing_lender_city|servicing_lender_state|servicing_lender_zip|
+----------------------------+---------------------+------------------------+---------------------+----------------------+--------------------+
|                       39197|  The Commercial Bank|           175 Hopper St|              DE KALB|                    MS|          39328-9808|
|                      107793| Washington State ...|        330 Union Ave SE|              OLYMPIA|                    WA|          98501-2062|
|                      122100| Neighborhood Nati...|         2987 Jamacha Rd|             EL CAJON|                    CA|               92019|
|                       96495|     Three Rivers FCU|     1615 Northland Blvd|           FORT WAYNE|                    IN|          4682

# Dim Date

In [23]:
def week_of_month(dt):
    year = dt.year
    month = dt.month
    day = dt.day
    
    cal = calendar.monthcalendar(year,month)
    week_number = (day - 1) // 7 + 1
    return week_number

    
#Create Dim
start_date = pd.to_datetime("2017-01-01")
end_date = pd.to_datetime("2022-12-31")

dim_date = pd.DataFrame({"date": pd.date_range(start_date, end_date, freq="D")})

dim_date['date_id'] = dim_date['date'].dt.strftime('%Y%m%d')
dim_date['year_number'] = dim_date['date'].dt.year
dim_date['month_number'] = dim_date['date'].dt.month
dim_date['day_number'] = dim_date['date'].dt.day
dim_date['week_number'] = dim_date['date'].dt.strftime('%V')
dim_date['week_of_month'] = dim_date['date'].apply(week_of_month)
dim_date['week_of_year'] = dim_date['date'].dt.strftime('%U')
dim_date['month_name'] = dim_date['date'].dt.strftime('%B')
dim_date['day_name'] = dim_date['date'].dt.strftime('%A')
dim_date['timestamp_isoformat'] = dim_date['date'].apply(lambda x: x.isoformat())

#Add to tables
tables.append([dim_date,"dim_date"])

#Show
#print(dim_date.shape)
#print(dim_date.info())
#print(dim_date.head())
dim_date.head()

,date,date_id,year_number,month_number,day_number,week_number,week_of_month,week_of_year,month_name,day_name,timestamp_isoformat
0,2017-01-01,20170101,2017,1,1,52,1,01,January,Sunday,2017-01-01T00:00:00
1,2017-01-02,20170102,2017,1,2,01,1,01,January,Monday,2017-01-02T00:00:00
2,2017-01-03,20170103,2017,1,3,01,1,01,January,Tuesday,2017-01-03T00:00:00
3,2017-01-04,20170104,2017,1,4,01,1,01,January,Wednesday,2017-01-04T00:00:00
4,2017-01-05,20170105,2017,1,5,01,1,01,January,Thursday,2017-01-05T00:00:00


# Dim Industry

In [24]:
naics_df

,2022 NAICS US Code,2022 NAICS US Title,industry_type
0,11,"Agriculture, Forestry, Fishing and Hunting","Agriculture, Forestry, Fishing and Hunting"
1,111,Crop Production,"Agriculture, Forestry, Fishing and Hunting"
2,1111,Oilseed and Grain Farming,"Agriculture, Forestry, Fishing and Hunting"
3,11111,Soybean Farming,"Agriculture, Forestry, Fishing and Hunting"
4,111110,Soybean Farming,"Agriculture, Forestry, Fishing and Hunting"
...,...,...,...
2120,9281,National Security and International Affairs,Public Administration
2121,92811,National Security,Public Administration
2122,928110,National Security,Public Administration
2123,92812,International Affairs,Public Administration


In [25]:
#Create Dim
dim_industry = pd.DataFrame()
dim_industry['NAICS_code'] = naics_df['2022 NAICS US Code']
dim_industry['industry_name'] = naics_df['2022 NAICS US Title']
dim_industry['industry_type'] = naics_df['industry_type']

#Add unique ID
dim_industry['industry_id'] = pd.factorize(dim_industry['NAICS_code'])[0] + 1

#Remove duplicates
dim_industry = dim_industry.drop_duplicates()

#Add to tables
tables.append([dim_industry,"dim_industry"])

#Show
dim_industry.head()

,NAICS_code,industry_name,industry_type,industry_id
0,11,"Agriculture, Forestry, Fishing and Hunting","Agriculture, Forestry, Fishing and Hunting",1
1,111,Crop Production,"Agriculture, Forestry, Fishing and Hunting",2
2,1111,Oilseed and Grain Farming,"Agriculture, Forestry, Fishing and Hunting",3
3,11111,Soybean Farming,"Agriculture, Forestry, Fishing and Hunting",4
4,111110,Soybean Farming,"Agriculture, Forestry, Fishing and Hunting",5


# Dim Census Location


In [26]:
#Create Dim
dim_census_location = ppp_df.select('ProjectCity','ProjectCountyName', 
                                    'ProjectState', 'ProjectZip','CD')
dim_cen_count_BEFORE = dim_census_location.count()

In [27]:
#Fix Names
dim_census_location = dim_census_location.withColumnRenamed("CD","cd")
dim_census_location = df_to_snake_case(dim_census_location)

#Add unique ID
dim_census_location = create_ids(dim_census_location, "census_location_id")

#Remove duplicates
dim_census_location = dim_census_location.distinct()

#Add to tables
tables.append([dim_census_location,"dim_census_location"])

#Show
dim_census_location.show()

Transforming PySpark DataFrame...
Completed transforming PySpark DataFrame.
Transforming PySpark DataFrame...


Completed transforming PySpark DataFrame.


+---------------+-------------------+-------------+-----------+-----+------------------+
|   project_city|project_county_name|project_state|project_zip|   cd|census_location_id|
+---------------+-------------------+-------------+-----------+-----+------------------+
|         SMYRNA|               COBB|           GA| 30080-3002|GA-11|               320|
|       MARIETTA|               COBB|           GA| 30068-4353|GA-06|              1995|
|         Warner|            HOUSTON|           GA| 31088-0300|GA-08|              2561|
|         ATHENS|             CLARKE|           GA| 30601-1952|GA-10|              2636|
|      MCDONOUGH|              HENRY|           GA| 30253-8034|GA-13|              2950|
|        EMERSON|             BARTOW|           GA| 30137-2219|GA-11|              4128|
|    STOCKBRIDGE|              HENRY|           GA| 30281-1196|GA-13|              4845|
|  Lawrenceville|           GWINNETT|           GA| 30046-5753|GA-07|              4950|
|       TAMUNING|    

# Facts GDP

In [28]:
#Create facts
facts_gdp = gdp_df

#Fix names
facts_gdp = df_to_snake_case(facts_gdp)
facts_gdp.rename(columns={'geo_f_i_p_s':'geofips'}, inplace=True)

#Remove duplicates
facts_gdp = facts_gdp.drop_duplicates()

#Drop unnecessary columns
facts_gdp.drop('industry_classification',axis=1,inplace=True)

Transforming Pandas DataFrame...
Completed transforming Pandas DataFrame.


In [29]:
# Convert columns 2017-2022 to float
columns_to_convert = ['2017', '2018', '2019', '2020', '2021', '2022']

# Convert columns to float while replacing invalid entries with NaN
facts_gdp[columns_to_convert] = facts_gdp[columns_to_convert].apply(pd.to_numeric, errors='coerce')

In [30]:
#Show
facts_gdp

,geofips,geo_name,region,table_name,line_code,description,unit,2017,2018,2019,2020,2021,2022
0,1000,Alabama,5.0,CAGDP1,1,Real GDP (thousands of chained 2017 dollars),Thousands of chained 2017 dollars,2.166155e+08,2.208088e+08,2.249446e+08,2.220814e+08,2.318926e+08,2.358073e+08
1,1000,Alabama,5.0,CAGDP1,2,Chain-type quantity indexes for real GDP,Quantity index,1.000000e+02,1.019360e+02,1.038450e+02,1.025230e+02,1.070530e+02,1.088600e+02
2,1000,Alabama,5.0,CAGDP1,3,Current-dollar GDP (thousands of current dolla...,Thousands of dollars,2.166155e+08,2.262638e+08,2.345264e+08,2.351183e+08,2.579865e+08,2.815690e+08
3,1001,"Autauga, AL",5.0,CAGDP1,1,Real GDP (thousands of chained 2017 dollars),Thousands of chained 2017 dollars,1.762558e+06,1.787534e+06,1.730861e+06,1.722438e+06,1.727818e+06,1.929264e+06
4,1001,"Autauga, AL",5.0,CAGDP1,2,Chain-type quantity indexes for real GDP,Quantity index,1.000000e+02,1.014170e+02,9.820200e+01,9.772400e+01,9.802900e+01,1.094580e+02
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9526,97000,Rocky Mountain,7.0,CAGDP1,2,Chain-type quantity indexes for real GDP,Quantity index,1.000000e+02,1.045590e+02,1.095520e+02,1.093290e+02,1.166610e+02,1.193240e+02
9527,97000,Rocky Mountain,7.0,CAGDP1,3,Current-dollar GDP (thousands of current dolla...,Thousands of dollars,6.813101e+08,7.305677e+08,7.762811e+08,7.812724e+08,8.801425e+08,9.746826e+08
9528,98000,Far West,8.0,CAGDP1,1,Real GDP (thousands of chained 2017 dollars),Thousands of chained 2017 dollars,3.797440e+09,3.956948e+09,4.108526e+09,4.048650e+09,4.342903e+09,4.385658e+09
9529,98000,Far West,8.0,CAGDP1,2,Chain-type quantity indexes for real GDP,Quantity index,1.000000e+02,1.042000e+02,1.081920e+02,1.066150e+02,1.143640e+02,1.154900e+02


#### Now we will begin the process of connecting facts_gdp with dim_census_location

In [31]:
# Start by converting facts_gdp to a spark df so we can work on it better
facts_gdp_spark = spark.createDataFrame(facts_gdp)

In [32]:
#Next, we split the geo_name column so we can easily get the county name and state abbriviation
facts_gdp_spark = facts_gdp_spark.withColumn("county_name", split(col("geo_name"), ",")[0])
facts_gdp_spark = facts_gdp_spark.withColumn("state_abbreviation", trim(split(col("geo_name"), ",")[1]))
facts_gdp_spark.select("geo_name","county_name","state_abbreviation").orderBy(rand()).limit(10).show()

+--------------------+------------------+------------------+
|            geo_name|       county_name|state_abbreviation|
+--------------------+------------------+------------------+
|       St. Johns, FL|         St. Johns|                FL|
|           Lamar, TX|             Lamar|                TX|
|        Stephens, OK|          Stephens|                OK|
|       Habersham, GA|         Habersham|                GA|
|          Gibson, TN|            Gibson|                TN|
|           Essex, VA|             Essex|                VA|
|    King William, VA|      King William|                VA|
|        Montague, TX|          Montague|                TX|
|Petersburg Boroug...|Petersburg Borough|               AK*|
|         Andrews, TX|           Andrews|                TX|
+--------------------+------------------+------------------+



In [33]:
# quickly get the count of rows before we set the df to the joined df. This is for debugging purposes
facts_gpd_spark_before_count = facts_gdp_spark.count()

In [34]:
# Clean the state abbreviations (e.g., removing any "*" characters)
df1_cleaned = facts_gdp_spark.withColumn("state_abbreviation", regexp_replace("state_abbreviation", r"\*", ""))
df2_cleaned = dim_census_location.withColumn("project_state", regexp_replace("project_state", r"\*", ""))

# Make join condition (which is also case insensitive)
join_condition = (lower(col("df1.county_name")) == lower(col("df2.project_county_name"))) & \
                 (lower(col("df1.state_abbreviation")) == lower(col("df2.project_state")))

# Perform the join
joined_df = df1_cleaned.alias("df1").join(df2_cleaned.alias("df2"), on=join_condition, how="left")

# Show the result
joined_df.select("county_name","state_abbreviation","project_county_name",
                 "project_state","project_zip","cd", "census_location_id").orderBy(rand()).limit(10).show()

+------------+------------------+-------------------+-------------+-----------+-----+------------------+
| county_name|state_abbreviation|project_county_name|project_state|project_zip|   cd|census_location_id|
+------------+------------------+-------------------+-------------+-----------+-----+------------------+
|Contra Costa|                CA|       CONTRA COSTA|           CA| 94523-0001|CA-10|           7129229|
| Los Angeles|                CA|        LOS ANGELES|           CA| 91307-2652|CA-32|           5274745|
| Bartholomew|                IN|        BARTHOLOMEW|           IN| 47244-9772|IN-06|           3661941|
|       Davis|                UT|              DAVIS|           UT| 84087-1038|UT-02|           4123402|
|      Nassau|                NY|             NASSAU|           NY| 11753-0001|NY-03|           8303927|
|    Hartford|                CT|           HARTFORD|           CT| 06110-0001|CT-01|            730555|
|       Henry|                OH|              HENRY|  

In [35]:
# Set facts_gdp_spark to new joined_df, then delete joined df
facts_gdp_spark = joined_df
del joined_df

In [36]:
# Finally, we drop columns thats not needed
facts_gdp_spark.columns
facts_gdp_spark = facts_gdp_spark.drop("county_name","state_abbreviation", "project_county_name",
                                      "project_state", "project_city","project_zip","cd")

In [37]:
# See how much we changed facts_gpd
facts_gpd_spark_after_count = facts_gdp_spark.count()
facts_gpd_pandas_count =len(facts_gdp)
dim_census_location_count = dim_census_location.count()

#Remove duplicates
facts_gdp_spark = facts_gdp_spark.distinct()

print("Fact_GPD_SPARK_BEFORE: "+str(facts_gpd_spark_before_count))
print("Fact_GPD_SPARK_AFTER: "+str(facts_gpd_spark_after_count))
print("Fact_GPD_PANDAS: "+str(facts_gpd_pandas_count))
print("DIM_Census_Location: "+str(dim_census_location_count))

Fact_GPD_SPARK_BEFORE: 9531
Fact_GPD_SPARK_AFTER: 29915703
Fact_GPD_PANDAS: 9531
DIM_Census_Location: 10406068


In [38]:
# Check how much of facts_gdp is null after join
!pip install handyspark
import numpy as np
np.bool = np.bool_
from handyspark import *
hsdf = HandyFrame(facts_gdp_spark)
hsdf.isnull()

/usr/lib/spark/python/pyspark/sql/dataframe.py:168: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(
/usr/lib/spark/python/pyspark/sql/dataframe.py:147: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


geofips                 0
geo_name                0
region                  0
table_name              0
line_code               0
description             0
unit                    0
2017                    0
2018                    0
2019                    0
2020                    0
2021                    0
2022                    0
census_location_id    513
Name: missing, dtype: int64

In [39]:
# Add unique ID
facts_gdp_spark = facts_gdp_spark.withColumn("facts_gdp_id", monotonically_increasing_id()+1)

# Add to tables
tables.append([facts_gdp_spark,"facts_gdp"])

# Facts PPP Loans

In [41]:
# Small function to convert date columns to date_id to connect to dim_date table.
def date_to_id(df, column_name):
    df = df.withColumn(column_name+"_id", date_format(column_name, "yyyyMMdd").cast("int"))
    df = df.drop(column_name)
    return df


# Create facts
facts_PPP_loans = ppp_df.select("SBAGuarantyPercentage","InitialApprovalAmount","CurrentApprovalAmount",
                               "UndisbursedAmount","ForgivenessAmount","LoanNumber","LoanStatus","Term",
                               "RuralUrbanIndicator","HubzoneIndicator","LMIIndicator","JobsReported",
                                "date_approved", "loan_status_date","forgiveness_date",
                               "SBAOfficeCode","ServicingLenderLocationID","OriginatingLenderLocationID",
                               "ProcessingMethod","NAICSCode")

# Convert date cols to date_ids so we can connect it to the dim date if needed
facts_PPP_loans = date_to_id(facts_PPP_loans,"date_approved")
facts_PPP_loans = date_to_id(facts_PPP_loans,"loan_status_date")
facts_PPP_loans = date_to_id(facts_PPP_loans,"forgiveness_date")


# Fix names
facts_PPP_loans = df_to_snake_case(facts_PPP_loans)
facts_PPP_loans = facts_PPP_loans.withColumnRenamed("s_b_a_office_code","sba_office_code")
facts_PPP_loans = facts_PPP_loans.withColumnRenamed("s_b_a_guaranty_percentage","sba_guaranty_percentage")
facts_PPP_loans = facts_PPP_loans.withColumnRenamed("servicing_lender_location_i_d","servicing_lender_location_id")
facts_PPP_loans = facts_PPP_loans.withColumnRenamed("originating_lender_location_i_d","originating_lender_location_id")
facts_PPP_loans = facts_PPP_loans.withColumnRenamed("l_m_i_indicator","lmi_indicator")
facts_PPP_loans = facts_PPP_loans.withColumnRenamed("n_a_i_c_s_code","NAICS_code")


# Add foreign IDs
facts_PPP_loans = facts_PPP_loans.withColumn("borrower_id", col("loan_number")) 

# Adding the industry_id as a foreign key.
#- 1st, join facts dataframe with dim_industry on the naics code
facts_PPP_loans = facts_PPP_loans.join(spark.createDataFrame(dim_industry), 
                                       on="NAICS_code", how="inner")
#- 2nd, drop all columns from dim_industry beside the industry id
facts_PPP_loans = facts_PPP_loans.drop("NAICS_code","industry_type","industry_name")

#- 3rd, rename industry_id to NAICS_code_industry_id
facts_PPP_loans = facts_PPP_loans.withColumnRenamed("industry_id","NAICS_code")

# Add unique IDs
facts_PPP_loans = facts_PPP_loans.withColumn("fact_ppp_id", monotonically_increasing_id()+1)

#Remove duplicates
facts_PPP_loans = facts_PPP_loans.distinct()

# Add to tables
tables.append([facts_PPP_loans,"facts_PPP_loans"])

# Show
facts_PPP_loans.columns

Transforming PySpark DataFrame...
Completed transforming PySpark DataFrame.


['sba_guaranty_percentage',
 'initial_approval_amount',
 'current_approval_amount',
 'undisbursed_amount',
 'forgiveness_amount',
 'loan_number',
 'loan_status',
 'term',
 'rural_urban_indicator',
 'hubzone_indicator',
 'lmi_indicator',
 'jobs_reported',
 'sba_office_code',
 'servicing_lender_location_id',
 'originating_lender_location_id',
 'processing_method',
 'date_approved_id',
 'loan_status_date_id',
 'forgiveness_date_id',
 'borrower_id',
 'NAICS_code',
 'fact_ppp_id']

# Functions for saving

Note: All functions assume we have global variable "bucket" that contains the name of the bucket we are working with

In [42]:
def check_folder_exists(folder_name):
    
    # Grab the bucket
    client = storage.Client()
    bucket_ = client.get_bucket(bucket)
    
    # List objects in the bucket with the folder prefix
    blobs = bucket_.list_blobs(prefix=folder_name + '/')
    
    # Check if any blob exists under this folder
    for blob in blobs:
        if blob.name.startswith(folder_name + '/'):
            print(f"Folder '{folder_name}' exists in bucket '{bucket}'.")
            return True
    
    print(f"Folder '{folder_name}' does not exist in bucket '{bucket}'.")
    return False

In [43]:
def create_folder(path, name):
    
    if check_folder_exists(name): 
        print(path+name+'/'+' already exists. Abandoning folder creation')
        return
    
    # Initialize the GCS client
    client = storage.Client()

    # Get the bucket
    _bucket = client.get_bucket(bucket)

    # Define the folder path
    folder_path = path+name+'/'

    # Create a dummy file in the folder (this will simulate the folder in GCS)
    # The 'blob' will not be a real file, but will create the "folder"
    blob = _bucket.blob(folder_path + 'placeholder.txt')  # You can name it anything, like 'placeholder.txt'

    # Upload an empty string or any content to simulate the folder creation
    blob.upload_from_string('')

    print(f"Folder '{folder_path}' created successfully in the bucket '{bucket}'")

In [44]:
def delete_files_named(name_of_file_to_delete):
    # Initialize a client
    client = storage.Client()

    # Specify the bucket name
    _bucket = client.bucket(bucket)

    # List and delete all files named "placeholder.txt"
    blobs = _bucket.list_blobs()

    count_deleted = 0
    for blob in blobs:
        if blob.name.endswith(name_of_file_to_delete):
            blob.delete()
            print(f"Deleted: {blob.name}")
            count_deleted += 1
            
    
    print(f"Deletion of all '{name_of_file_to_delete}' files is complete. Total deleted: "+str(count_deleted))

In [45]:
def save_df(df,name,save_as="parquet",gcs_path=None):

    #we must define the bucket variable at the start of the script, before calling this function.
    
    # Define the GCS path for saving the file
    if gcs_path == None:
        gcs_path = 'gs://'+bucket+'/dim_ready/'+name+'/'+name+'.'+save_as
    
    
    
    print(f"Creating '{name}' folder...")
    create_folder('dim_ready/',name)
    delete_files_named("placeholder.txt")
    
    print("Beginning saving process...")
    flag = 0 # this var will let us know is there as been an error
    
    #Saves Pandas dataframe
    if isinstance(df, pd.DataFrame):
        # Initialize GCS file system
        fs = gcsfs.GCSFileSystem(project='cis4400-group-project')

        # Save DataFrame to the "cleaned" folder in the GCS bucket
        with fs.open(gcs_path, 'w') as f:
            if save_as == "csv":
                df.to_csv(f, index=False)
            else:
                print("INVALD FILE FORMAT. TRY [csv]")
                flag+=1
        
        if flag == 0:
            print("Successfully saved "+name+ "!")
    
    #Saves PySpark dataframe
    elif isinstance(df, PySparkDataFrame):
        if save_as == "parquet":
            df.write.parquet(gcs_path,mode="overwrite")
        elif save_as == "csv":
            df.write.csv(gcs_path,mode="overwrite")
        elif save_as == "avro":
            df.write.format("avro").save(gcs_path,mode="overwrite")
        else:
            print("INVALD FILE FORMAT. TRY [csv] or [parquet] or [avro]")
            flag+=1
        
        if flag == 0:
            print("Successfully saved "+name+ "!")
        
    #No valid dataframe found
    else:
        print("ERROR: INVALID DATAFRAME - NO PROCEDURE APPLIED")

In [46]:
#this function accepts a list of tables, that each contain 2 elements, the table itself, and the name of the table.
#Example, table[0] = [df_1,"df_1_name"], table[1][0] = df_2, table[3][1] = "df_4_name"
def save_list_of_df(list_of_tables,pandas_save="csv",pyspark_save="parquet"):
    for table in list_of_tables:
        if isinstance(table[0], pd.DataFrame):
            save_df(table[0],table[1],save_as=pandas_save)
        elif isinstance(table[0], PySparkDataFrame):
            save_df(table[0],table[1],save_as=pyspark_save)

# Saving Dims and Facts

In [47]:
#We will simply save all Pandas dataframes as CSV's and all PySpark ones as parquet

ENABLE_SAVE = True #boolean to quickly turn on/off saving. Good to turn off saving when debugging.

if ENABLE_SAVE:
    save_list_of_df(tables)

Creating 'dim_borrower' folder...
Folder 'dim_borrower' does not exist in bucket 'ppp-loans-bucket'.
Folder 'dim_ready/dim_borrower/' created successfully in the bucket 'ppp-loans-bucket'
Deleted: dim_ready/dim_borrower/placeholder.txt
Deletion of all 'placeholder.txt' files is complete. Total deleted: 1
Beginning saving process...


Successfully saved dim_borrower!
Creating 'dim_SBA_office' folder...
Folder 'dim_SBA_office' does not exist in bucket 'ppp-loans-bucket'.
Folder 'dim_ready/dim_SBA_office/' created successfully in the bucket 'ppp-loans-bucket'
Deleted: dim_ready/dim_SBA_office/placeholder.txt
Deletion of all 'placeholder.txt' files is complete. Total deleted: 1
Beginning saving process...
Successfully saved dim_SBA_office!
Creating 'dim_processing_method' folder...
Folder 'dim_processing_method' does not exist in bucket 'ppp-loans-bucket'.
Folder 'dim_ready/dim_processing_method/' created successfully in the bucket 'ppp-loans-bucket'
Deleted: dim_ready/dim_processing_method/placeholder.txt
Deletion of all 'placeholder.txt' files is complete. Total deleted: 1
Beginning saving process...
Successfully saved dim_processing_method!
Creating 'dim_originating_lender' folder...
Folder 'dim_originating_lender' does not exist in bucket 'ppp-loans-bucket'.
Folder 'dim_ready/dim_originating_lender/' created succes

Successfully saved dim_originating_lender!
Creating 'dim_servicing_lender' folder...
Folder 'dim_servicing_lender' does not exist in bucket 'ppp-loans-bucket'.
Folder 'dim_ready/dim_servicing_lender/' created successfully in the bucket 'ppp-loans-bucket'
Deleted: dim_ready/dim_servicing_lender/placeholder.txt
Deletion of all 'placeholder.txt' files is complete. Total deleted: 1
Beginning saving process...


Successfully saved dim_servicing_lender!
Creating 'dim_date' folder...
Folder 'dim_date' does not exist in bucket 'ppp-loans-bucket'.
Folder 'dim_ready/dim_date/' created successfully in the bucket 'ppp-loans-bucket'
Deleted: dim_ready/dim_date/placeholder.txt
Deletion of all 'placeholder.txt' files is complete. Total deleted: 1
Beginning saving process...
Successfully saved dim_date!
Creating 'dim_industry' folder...
Folder 'dim_industry' does not exist in bucket 'ppp-loans-bucket'.
Folder 'dim_ready/dim_industry/' created successfully in the bucket 'ppp-loans-bucket'
Deleted: dim_ready/dim_industry/placeholder.txt
Deletion of all 'placeholder.txt' files is complete. Total deleted: 1
Beginning saving process...
Successfully saved dim_industry!
Creating 'dim_census_location' folder...
Folder 'dim_census_location' does not exist in bucket 'ppp-loans-bucket'.
Folder 'dim_ready/dim_census_location/' created successfully in the bucket 'ppp-loans-bucket'
Deleted: dim_ready/dim_census_locati

Successfully saved dim_census_location!
Creating 'facts_gdp' folder...
Folder 'facts_gdp' does not exist in bucket 'ppp-loans-bucket'.
Folder 'dim_ready/facts_gdp/' created successfully in the bucket 'ppp-loans-bucket'
Deleted: dim_ready/facts_gdp/placeholder.txt
Deletion of all 'placeholder.txt' files is complete. Total deleted: 1
Beginning saving process...


Successfully saved facts_gdp!
Creating 'facts_PPP_loans' folder...
Folder 'facts_PPP_loans' does not exist in bucket 'ppp-loans-bucket'.
Folder 'dim_ready/facts_PPP_loans/' created successfully in the bucket 'ppp-loans-bucket'
Deleted: dim_ready/facts_PPP_loans/placeholder.txt
Deletion of all 'placeholder.txt' files is complete. Total deleted: 1
Beginning saving process...


Successfully saved facts_PPP_loans!


In [48]:
# DIFFERENCES BETWEEN THIS AND CURRENT DIM MODEL

# PPP Fact Table currently has "term" and "term_date_id" in the model. However...
# Script wise, I did not include term_date_id, and left term as is since it doesnt correspond to a date.